# Item Response Theory

This notebook executes the experiments for Item Response Theory, which is performed both with and without reference classes.

The experiments for WRC are separated into 3 versions (1, 3, 4).
The experiments for WORC are all performed in version 2.

Note: The code in this notebook for the different versions and parts only differs by the specified class-to-reference-class dictionary and the conf dictionary used.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime

import sys
import os
sys.path.append(os.path.abspath('../../sources'))

import config
import utils
import training_general
import training_with_rc
import training_without_rc
from data_preparation import determine_reference_classes

## WITH REFERENCE CLASS

In [ ]:
# read data
df = utils.read_data_file("final_data_main_approach.csv")
df_orig = df.copy()
print(df.shape)

# get dictionary with reference classes
class_to_reference_class = determine_reference_classes.get_reference_classes(df)

# uncomment when executing version 4
#class_to_reference_class = determine_reference_classes.restrict_c2rc_dict(
#    class_to_reference_class, "ref_classes_restricted.csv"
#)

# uncomment when executing version 3
#class_to_reference_class = determine_reference_classes.restrict_c2rc_dict(
#    class_to_reference_class, "ref_classes_restricted_random.csv"
#)

In [ ]:
# separate class_to_reference_class in 4 parts
cut1 = 350
cut2 = 600
cut3 = 1050
c2rc_part1 = {k: v for k, v in list(class_to_reference_class.items())[:cut1]}
c2rc_part2 = {k: v for k, v in list(class_to_reference_class.items())[cut1:cut2]}
c2rc_part3 = {k: v for k, v in list(class_to_reference_class.items())[cut2:cut3]}
c2rc_part4 = {k: v for k, v in list(class_to_reference_class.items())[cut3:]}
print(len(c2rc_part1), len(c2rc_part2), len(c2rc_part3), len(c2rc_part4))

### Version 1: mean / mean --> all reference classes

In [ ]:
def get_conf_version1(filename_suffix: str) -> dict:
    return {
        "lim": [0.3, 0.5, 0.7, config.LimType.DYNAMIC],
        "eval_groups": ["info_cols", "reg_metrics", "class_metrics"],
        "reg_metrics": [config.RegMetrics.MAE, config.RegMetrics.MSE],
        "class_metrics": [
            config.ClassMetrics.ACC,
            config.ClassMetrics.F1,
            config.ClassMetrics.PREC,
            config.ClassMetrics.REC,
        ],
        "info_cols": [
            config.InfoCols.NUM_UT_PROBS,
            config.InfoCols.NUM_IU_PROBS,
            config.InfoCols.MEAN_UT_PERF,
            config.InfoCols.MEAN_IU_PERF,
            config.InfoCols.NUM_STUD_RC,
            config.InfoCols.MAX_NUM_IU_PROBS_RC,
            config.InfoCols.MEAN_IU_PERF_RC,
            config.InfoCols.MEAN_UT_PERF_RC
        ],
        "method": config.RecMethod.IRT,
        "with_ref_class": True,
        "models": [
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "difficulty": "mean",
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "ab_int": 3,
                "difficulty": "mean",
                "diff_int": 3,
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "ab_int": 3,
                "difficulty": "mean",
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "difficulty": "mean",
                "diff_int": 3,
            },
        ],
        "saving_file": {
            "folder": "item_response_theory",
            "filename": "version1",
            "filename_suffix": filename_suffix,
        },
    }

save_file = True

##### Part 1

In [ ]:
c2rc = c2rc_part1.copy()
df = df_orig.copy()

conf = get_conf_version1(filename_suffix="part1")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

In [ ]:
# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:1]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df1 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df1.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 2

In [ ]:
c2rc = c2rc_part2.copy()
df = df_orig.copy()

conf = get_conf_version1(filename_suffix="part2")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df2 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df2.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 3

In [ ]:
c2rc = c2rc_part3.copy()
df = df_orig.copy()

conf = get_conf_version1(filename_suffix="part3")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df3 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df3.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 4

In [ ]:
c2rc = c2rc_part4.copy()
df = df_orig.copy()

conf = get_conf_version1(filename_suffix="part4")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df4 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df4.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

### Version 4: Proportion Correct and Elo --> restricted reference classes

In [ ]:
def get_conf_version4(filename_suffix: str) -> dict:
    return {
        "lim": [0.5],
        #"lim": [0.3, 0.5, 0.7, config.LimType.DYNAMIC],
        "eval_groups": ["info_cols", "reg_metrics", "class_metrics"],
        "reg_metrics": [config.RegMetrics.MAE, config.RegMetrics.MSE],
        "class_metrics": [
            config.ClassMetrics.ACC,
            config.ClassMetrics.F1,
            config.ClassMetrics.PREC,
            config.ClassMetrics.REC,
        ],
        "info_cols": [
            config.InfoCols.NUM_UT_PROBS,
            config.InfoCols.NUM_IU_PROBS,
            config.InfoCols.MEAN_UT_PERF,
            config.InfoCols.MEAN_IU_PERF,
            config.InfoCols.NUM_STUD_RC,
            config.InfoCols.MAX_NUM_IU_PROBS_RC,
            config.InfoCols.MEAN_IU_PERF_RC,
            config.InfoCols.MEAN_UT_PERF_RC
        ],
        "method": config.RecMethod.IRT,
        "with_ref_class": True,
        "models": [
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "difficulty": "pc",
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "mean",
                "ab_int": 3,
                "difficulty": "pc",
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "elo",
                "difficulty": "elo",
                "W": 0.2,
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "elo",
                "difficulty": "elo",
                "W": 0.4,
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "elo",
                "difficulty": "elo",
                "W": 0.6,
            },
        ],
        "saving_file": {
            "folder": "item_response_theory",
            "filename": "version4",
            "filename_suffix": filename_suffix,
        },
    }

save_file = True

##### Part 1

In [ ]:
c2rc = c2rc_part1.copy()
df = df_orig.copy()

conf = get_conf_version4(filename_suffix="part1")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

In [ ]:
# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:1]:
#for cid in ["2JFV80TTBO"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df1 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df1.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 2

In [ ]:
c2rc = c2rc_part2.copy()
df = df_orig.copy()

conf = get_conf_version4(filename_suffix="part2")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df2 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df2.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 3

In [ ]:
c2rc = c2rc_part3.copy()
df = df_orig.copy()

conf = get_conf_version4(filename_suffix="part3")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df3 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df3.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 4

In [ ]:
c2rc = c2rc_part4.copy()
df = df_orig.copy()

conf = get_conf_version4(filename_suffix="part4")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:3]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df4 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df4.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

### Run 3: py-irt --> restricted random reference classes

In [ ]:
def get_conf_version3(filename_suffix: str) -> dict:
    return {
        "lim": [0.3, 0.5, 0.7, config.LimType.DYNAMIC],
        "eval_groups": ["info_cols", "reg_metrics", "class_metrics"],
        "reg_metrics": [config.RegMetrics.MAE, config.RegMetrics.MSE],
        "class_metrics": [
            config.ClassMetrics.ACC,
            config.ClassMetrics.F1,
            config.ClassMetrics.PREC,
            config.ClassMetrics.REC,
        ],
        "info_cols": [
            config.InfoCols.NUM_UT_PROBS,
            config.InfoCols.NUM_IU_PROBS,
            config.InfoCols.MEAN_UT_PERF,
            config.InfoCols.MEAN_IU_PERF,
            config.InfoCols.NUM_STUD_RC,
            config.InfoCols.MAX_NUM_IU_PROBS_RC,
            config.InfoCols.MEAN_IU_PERF_RC,
            config.InfoCols.MEAN_UT_PERF_RC
        ],
        "method": config.RecMethod.IRT,
        "with_ref_class": True,
        "models": [
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "package",
                "difficulty": "1pl"
            },
            {
                "model_type": config.IRTModelType.WRC,
                "ability": "package",
                "difficulty": "2pl"
            }
        ],
        "saving_file": {
            "folder": "item_response_theory",
            "filename": "version3",
            "filename_suffix": filename_suffix,
        },
    }

save_file = True

##### Part 1

In [ ]:
c2rc = c2rc_part1.copy()
df = df_orig.copy()

conf = get_conf_version3(filename_suffix="part1")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

In [ ]:
# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:10]:
#for cid in ["2JFV80TTBO"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df1 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df1.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 2

In [ ]:
c2rc = c2rc_part2.copy()
df = df_orig.copy()

conf = get_conf_version3(filename_suffix="part2")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

In [ ]:
# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:10]:
#for cid in ["14MZLGKL6K"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df2 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df2.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 3

In [ ]:
c2rc = c2rc_part3.copy()
df = df_orig.copy()

conf = get_conf_version3(filename_suffix="part3")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:10]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df3 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df3.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

##### Part 4

In [ ]:
c2rc = c2rc_part4.copy()
df = df_orig.copy()

conf = get_conf_version3(filename_suffix="part4")

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# create dataframes
df, ass_seq, stud_per_class = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    index = training_with_rc.get_idx_pred_df(c2rc)
else:
    raise NotImplementedError
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
for cid, cid_dict in c2rc.items():
#for cid, cid_dict in list(c2rc.items())[:10]:
#for cid in ["ODWI4QBKV"]:
    #cid_dict = c2rc[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        raise NotImplementedError

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df4 = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df4.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)

## WITHOUT REFERENCE CLASS

In [ ]:
# read data
df = utils.read_data_file("final_data_main_approach.csv")
df_orig = df.copy()
print(df.shape)

### Version 2: Without Reference Classes

In [ ]:
def get_conf_version2(filename_suffix: str) -> dict:
    return {
        "lim": [0.3, 0.5, 0.7, config.LimType.DYNAMIC],
        "eval_groups": ["info_cols", "reg_metrics", "class_metrics"],
        "reg_metrics": [config.RegMetrics.MAE, config.RegMetrics.MSE],
        "class_metrics": [
            config.ClassMetrics.ACC,
            config.ClassMetrics.F1,
            config.ClassMetrics.PREC,
            config.ClassMetrics.REC,
        ],
        "info_cols": [
            config.InfoCols.NUM_UT_PROBS,
            config.InfoCols.NUM_IU_PROBS,
            config.InfoCols.MEAN_UT_PERF,
            config.InfoCols.MEAN_IU_PERF,
        ],
        "method": config.RecMethod.IRT,
        "with_ref_class": False,
        "models": [
            {
                "model_type": config.IRTModelType.WORC,
                "ability": "mean",
                "difficulty": "expert",
            },
            {
                "model_type": config.IRTModelType.WORC,
                "ability": "mean",
                "ab_int": 3,
                "difficulty": "expert",
                "diff_int": 3,
            },
            {
                "model_type": config.IRTModelType.WORC,
                "ability": "mean",
                "difficulty": "expert",
                "diff_int": 3,
            },
            {
                "model_type": config.IRTModelType.WORC,
                "ability": "mean",
                "ab_int": 3,
                "difficulty": "expert",
            },
        ],
        "saving_file": {
            "folder": "item_response_theory",
            "filename": "version2",
            "filename_suffix": filename_suffix,
        },
    }

save_file = True

In [ ]:
conf = get_conf_version2(filename_suffix="")

df = df_orig.copy()

# check validity of conf dictionary
training_general.check_conf(conf, save_file=save_file)

with_rc = conf["with_ref_class"]

# get dictionary with reference classes
class_to_reference_class = determine_reference_classes.get_reference_classes(df)
print(len(class_to_reference_class))
# it is not used for the reference classes but to know which classes and test sequences are evaluated

# create dataframes
df, ass_seq, _ = training_general.create_dataframes(df)

# create empty evaluation dataframe for complete training
if with_rc:
    raise NotImplementedError
else:
    index = training_without_rc.get_idx_pred_df(class_to_reference_class)
pred_df = training_general.initialize_pred_df(index=index, conf=conf)


count = 0
#count = 320
for cid, cid_dict in class_to_reference_class.items():
#for cid, cid_dict in list(class_to_reference_class.items())[:2]:
#for cid in ["2JFV80TTBO"]:
    #cid_dict = class_to_reference_class[cid]
    #print(f"----------- Class {cid} ------------")

    # make predictions for cid, evaluate and store evaluation results
    if with_rc:
        pred_df.loc[cid] = (
            training_with_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq, stud_per_class
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )
    else:
        pred_df.loc[cid] = (
            training_without_rc.perform_predictions_for_cid(
                conf, cid, cid_dict, df, ass_seq
            )
            .reindex(pred_df.loc[cid].index)
            .to_numpy()
        )

    count += 1
    if count % 10 == 0:
        d = datetime.datetime.now()
        print(f"{count} classes completed, last cid: {cid}, time: {d}")

In [ ]:
# drop rows only containing nans
pred_df = pred_df.dropna(subset=["y_true"])
print(len(pred_df))
pred_df_wo = pred_df.copy()

# save
utils.save_predictions(pred_df, conf, save_idx=True)

In [ ]:
pred_df = pred_df_wo.copy()
# evaluate predictions and save
training_general.evaluate_predictions_and_save(pred_df, conf)